### 查看近兩個月費用

In [4]:
import boto3
from datetime import datetime, timedelta

def get_cost_and_usage(start_date, end_date):
    # 使用成本探查者（Cost Explorer）API
    client = boto3.client('ce')

    response = client.get_cost_and_usage(
        TimePeriod={
            'Start': start_date,
            'End': end_date
        },
        # 按月查詢
        Granularity='MONTHLY',
        # 查詢未混合成本
        Metrics=['UnblendedCost'],
        GroupBy=[
            {
                'Type': 'DIMENSION',
                'Key': 'SERVICE'
            }
        ]
    )
    return response

def print_cost_report(response, period_description):
    print(f"\nCost report for {period_description}:")
    for result in response['ResultsByTime']:
        print(f"Time Period: {result['TimePeriod']['Start']} - {result['TimePeriod']['End']}")
        if 'Groups' in result:
            for group in result['Groups']:
                service = group['Keys'][0]
                amount = group['Metrics']['UnblendedCost']['Amount']
                unit = group['Metrics']['UnblendedCost']['Unit']
                print(f"Service: {service}, Amount: {amount} {unit}")
        else:
            print("No cost data found for this time period.")

def main():
    # 定義查詢的時間範圍
    today = datetime.utcnow().date()
    start_of_this_month = today.replace(day=1).isoformat()
    start_of_last_month = (today.replace(day=1) - timedelta(days=1)).replace(day=1).isoformat()
    end_of_last_month = (today.replace(day=1) - timedelta(days=1)).isoformat()

    print(f"Querying cost data for the following periods:")
    print(f"Last month: {start_of_last_month} to {end_of_last_month}")
    print(f"This month to date: {start_of_this_month} to {today.isoformat()}")

    # 取得上個月的成本和使用情況
    response_last_month = get_cost_and_usage(start_of_last_month, end_of_last_month)
    print_cost_report(response_last_month, "Last month")

    # 取得本月至今的成本和使用情況
    response_this_month = get_cost_and_usage(start_of_this_month, today.isoformat())
    print_cost_report(response_this_month, "This month to date")

if __name__ == "__main__":
    main()


Querying cost data for the following periods:
Last month: 2024-06-01 to 2024-06-30
This month to date: 2024-07-01 to 2024-07-15

Cost report for Last month:
Time Period: 2024-06-01 - 2024-06-30

Cost report for This month to date:
Time Period: 2024-07-01 - 2024-07-15
Service: AWS CodeCommit, Amount: 0 USD
Service: EC2 - Other, Amount: 0.0000116589 USD
Service: Amazon Elastic Compute Cloud - Compute, Amount: 0 USD
Service: Amazon Elastic Load Balancing, Amount: 0.0000002151 USD
Service: Amazon Relational Database Service, Amount: 0.00690387 USD
Service: Amazon Simple Storage Service, Amount: 0.0000000045 USD
Service: Amazon Virtual Private Cloud, Amount: 0.072248615 USD
Service: AmazonCloudWatch, Amount: 0 USD
Service: Tax, Amount: 0 USD


### 合計費用，並從高到低排序

In [5]:
import boto3
from datetime import datetime, timedelta

def get_cost_and_usage(start_date, end_date):
    # 使用成本探查者（Cost Explorer）API
    client = boto3.client('ce')

    response = client.get_cost_and_usage(
        TimePeriod={
            'Start': start_date,
            'End': end_date
        },
        # 按月查詢
        Granularity='MONTHLY',
        # 查詢未混合成本
        Metrics=['UnblendedCost'],
        GroupBy=[
            {
                'Type': 'DIMENSION',
                'Key': 'SERVICE'
            }
        ]
    )
    return response

def print_cost_report(response, period_description):
    print(f"\nCost report for {period_description}:")
    total_cost = 0
    cost_data = []

    for result in response['ResultsByTime']:
        print(f"Time Period: {result['TimePeriod']['Start']} - {result['TimePeriod']['End']}")
        if 'Groups' in result:
            for group in result['Groups']:
                service = group['Keys'][0]
                amount = float(group['Metrics']['UnblendedCost']['Amount'])
                unit = group['Metrics']['UnblendedCost']['Unit']
                cost_data.append((service, amount, unit))
                total_cost += amount

    # 排序並打印費用數據
    cost_data.sort(key=lambda x: x[1], reverse=True)
    for service, amount, unit in cost_data:
        print(f"Service: {service}, Amount: {amount:.2f} {unit}")

    print(f"\nTotal Cost for {period_description}: {total_cost:.2f} USD")

def main():
    # 定義查詢的時間範圍
    today = datetime.utcnow().date()
    start_of_this_month = today.replace(day=1).isoformat()
    start_of_last_month = (today.replace(day=1) - timedelta(days=1)).replace(day=1).isoformat()
    end_of_last_month = (today.replace(day=1) - timedelta(days=1)).isoformat()

    print(f"Querying cost data for the following periods:")
    print(f"Last month: {start_of_last_month} to {end_of_last_month}")
    print(f"This month to date: {start_of_this_month} to {today.isoformat()}")

    # 取得上個月的成本和使用情況
    response_last_month = get_cost_and_usage(start_of_last_month, end_of_last_month)
    print_cost_report(response_last_month, "Last month")

    # 取得本月至今的成本和使用情況
    response_this_month = get_cost_and_usage(start_of_this_month, today.isoformat())
    print_cost_report(response_this_month, "This month to date")

if __name__ == "__main__":
    main()


Querying cost data for the following periods:
Last month: 2024-06-01 to 2024-06-30
This month to date: 2024-07-01 to 2024-07-15

Cost report for Last month:
Time Period: 2024-06-01 - 2024-06-30

Total Cost for Last month: 0.00 USD

Cost report for This month to date:
Time Period: 2024-07-01 - 2024-07-15
Service: Amazon Virtual Private Cloud, Amount: 0.07 USD
Service: Amazon Relational Database Service, Amount: 0.01 USD
Service: EC2 - Other, Amount: 0.00 USD
Service: Amazon Elastic Load Balancing, Amount: 0.00 USD
Service: Amazon Simple Storage Service, Amount: 0.00 USD
Service: AWS CodeCommit, Amount: 0.00 USD
Service: Amazon Elastic Compute Cloud - Compute, Amount: 0.00 USD
Service: AmazonCloudWatch, Amount: 0.00 USD
Service: Tax, Amount: 0.00 USD

Total Cost for This month to date: 0.08 USD
